In [1]:
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
import obspy
from obspy.taup import TauPyModel
import os
import glob
from rfpy import RFData
from os.path import exists
import numpy as np
import shutil
model = TauPyModel(model="iasp91")
client = Client("IRIS")
import matplotlib.pyplot as plt
from matplotlib.dates import num2date
import pandas as pd
plt.style.use('ggplot')


#############################################################################################################################################
# This class is used to do Quality Control over RFs created using make_rf class
#############################################################################################################################################
class rf_qc():
    """
    This class is used to perform quality control on receiver functions. 
    to initiate the class, the user must provide a dictionary of input values.
    input_values_for_qc
    """
#############################################################################################################################################
    def __init__(self, input_values_for_qc):
        self.db_name = input_values_for_qc['db_name']
        self.sorting_qc_list = input_values_for_qc['sorting_qc_list']
        self.hist_bin_number = input_values_for_qc['hist_bin_number']
        self.number_snr_plots = input_values_for_qc['number_snr_plots']
        if self.number_snr_plots%2 != 0:  #to ensure this value is always even
            self.number_snr_plots += 1
        
        self.mk_QC_list()



#############################################################################################################################################
    def mk_QC_list(self):
        path = f"{self.db_name}/02_detail.txt"
        
        if not os.path.isfile(path):
            raise Exception(f"File {path} does not exist. This file should be created using 'make_rf' class.")
        
        with open(path, "r") as f:
            details = f.read().splitlines()
        detail_length = len(details)

        QC_list = []
        for counter, each_line in enumerate(details):
            path = each_line.split(" ")[0]
            if each_line.split(" ")[1] == "None":
                continue
            snr_Z = float(each_line.split(" ")[1])
            snr_R = float(each_line.split(" ")[2])
            P_phase = each_line.split(" ")[3]
            BHorHH = glob.glob(f"{path}/raw*")[0].split("/")[-1].split("_")[0].split(("."))[1][0:2]
            delimiter = "--"
            sta = path.split("/")[1]
            QC_QUAL = 0                                                                   # 0: not QCed, 1-3: QCed based on the quality
            QC_list.append([path, snr_Z, snr_R, sta, delimiter, QC_QUAL, P_phase, BHorHH])           # print(path, snr_Z, snr_R, sta, delimiter, QC_by_eye, P_phase, BHorHH)        
            self.show_percentage_done(counter, detail_length)

        self.QC_list = pd.DataFrame(QC_list, columns=["path", "snr_Z", "snr_R", "NET_STA", "--", "QC_QUAL", "P_time", "BHorHH"])
        


#############################################################################################################################################
    def show_percentage_done(self, for_loop_counter, variable_length):
        # this function is used to show the percentage of the job done in the for loop
        # for_loop_counter: the counter of the for loop
        # variable_length: the length of the variable that is used in the for loop
        # this function is needed to be used inside the for loop which has a counter or enumerate

        print(f"QC list progress: {(for_loop_counter/variable_length*100)//1+1}% is completed.", end="\r")


#############################################################################################################################################
    def snr_hist(self):
        # this function is used to plot the histogram of SNR of the receiver function
        #you can change bin_number to see the histogram with different bin numbers.
        snr_list = self.QC_list[self.sorting_qc_list].tolist()
        plt.hist(snr_list, bins=self.hist_bin_number)
        plt.xlabel("SNR")
        plt.ylabel("Count")
        plt.show()


#############################################################################################################################################
    def visualize_snr(self):
        # this function is used to visualize the SNR of the receiver function to see what would be the cutoff value for SNR
        #we seperate the RFs those have SNR available, but not "None".
        QC_list = self.QC_list[self.QC_list[self.sorting_qc_list] != "None"]

        #selecting the SNR values that we want to plot.
        max_snr = np.max(QC_list[self.sorting_qc_list].astype(float))
        selected_snr = np.linspace(1, max_snr*0.5, self.number_snr_plots)
        fig, axs = plt.subplots(nrows=int(self.number_snr_plots/2), ncols=2, figsize=(15, 14))
        axs = axs.flatten()

        #looping over the selected SNR values.
        counter = 0
        for snr in selected_snr:
            for line in QC_list.values:
                if np.abs(float(line[2])-snr) < 1:
                    
                #plotting the RFs
                    tr = obspy.read(f"{line[0]}/RFR.sac")
                    tr.filter("bandpass", freqmin=0.01, freqmax=0.5)
                    tr.normalize()
                    time = tr[0].times("matplotlib")
                    data = tr[0].data

                #plotting the data       
                    axs[counter].plot(num2date(time), data,linewidth=0.45)
                    axs[counter].xaxis_date()
                    axs[counter].set_title(f"SNR: {round(line[2],0)} /\ {tr[0].stats.sac.kcmpnm}")
                    axs[counter].set_ylabel("Amplitude")
                    axs[counter].set_xlim([time[0], time[-1]])
                                   
                    counter += 1
                    break
        axs[len(selected_snr)-1].set_xlabel("Time (s)") 
        axs[len(selected_snr)-2].set_xlabel("Time (s)") 

        plt.tight_layout()


#############################################################################################################################################
    def snr_threshold_report(self, snr_threshold):
        # this function is used to report the number of RFs that has a SNR value higher than threshold
        # snr_threshold: the threshold value for SNR
        self.snr_threshold = snr_threshold
        
        self.QC_list = self.QC_list[self.QC_list[self.sorting_qc_list] > snr_threshold]
        
        QC_list_grouped = self.QC_list.groupby("NET_STA").count()["path"]

        with open(f"{self.db_name}/03_snr_report.txt", "w") as f:
            f.write(f"SNR threshold: {snr_threshold}\n")
            f.write(f"Total number of events: {QC_list_grouped.sum()}\n")
            f.write(f"Number of events per station:\n")
            f.write(f"{QC_list_grouped}\n")
        self.QC_list.to_csv(f"{self.db_name}/03_QC_list.tmp", index=False)
                    
    
#############################################################################################################################################
    
    def QC_by_eye(self):
        # !!!!!!!! this function is being able to be run individually after intiating the class.
        # this function is used to perform the quality control by eye.
        # the user can go through the RFs and decide if the RF is good or not.      
        try:
            QC_list = pd.read_csv(f"{self.db_name}/03_QC_list.txt")
            print("working on previous QC list...")
        except:
            QC_list = pd.read_csv(f"{self.db_name}/03_QC_list.tmp")
            
        QC_list = self.QC_eye_loop(QC_list)
        QC_list.to_csv(f"{self.db_name}/03_QC_list.txt", index=False)
        print("percentage of remaining:" ,len(QC_list[QC_list["QC_QUAL"]==0])/len(QC_list)*100)
            

#############################################################################################################################################      
    def QC_eye_loop(self, QC_list):
        #QC_list_local is the list of RFs that have SNR higher than the threshold value.
        
        for index, values in QC_list.iterrows():
            path = values["path"]
            if values["QC_QUAL"] in ["0", 0]:
                obspy.read(f"{path}/RFR.sac").plot()
                qc_value = input("QC value: (1: low, 2: Medium, 3:high) ---- (press 'q' to exit):")
                if qc_value in ["1", "2", "3"]:
                    QC_list["QC_QUAL"][index] = qc_value
                elif qc_value in ["q", "exit"]:
                    break
                else:
                    QC_list["QC_QUAL"][index] = "1"

        return QC_list
     

In [2]:
input_values_for_qc = {"db_name": "DB_CN",
                       "sorting_qc_list": "snr_R",
                       "hist_bin_number": 14,
                       "number_snr_plots": 2
                       }

Nunavut_project_qc = rf_qc(input_values_for_qc)
# Nunavut_project_qc.snr_hist()
# Nunavut_project_qc.visualize_snr()
Nunavut_project_qc.snr_threshold_report(7)   #here you can change the threshold


In [3]:
Nunavut_project_qc.QC_by_eye()

working on previous QC list...
percentage of remaining: 0.0
